# Check labels

Due to a change in the PDF Report paragraph splitting and the keyword list updates, there might be some inconsistencies (or overlapping labels)
between negative and positive labels. As a strategy: Remove overlapping labels if they are from the same page!

- Loop through neg labels and check if it is contained in positive labels

In [2]:
import pandas as pd
import numpy as np

# Read training and test data
df_train = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training.pkl")
df_test = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Test.pkl")


In [11]:
df_train.groupby(['cro', 'cro_sub_type_combined']).count()

report_id  cro_sub_type  page  paragraph_no  label  \
cro cro_sub_type_combined                                                       
OP  PRODUCTS                     423           423   423           423    403   
    RESILIENCE                   423           423   423           423    413   
PR  ACUTE                        113           113   113           113    110   
    CHRON                         21            21    21            21     21   
TR  MARKET                        51            51    51            51     46   
    POLICY                       109           109   109           109     99   
    REPUTATION                    29            29    29            29     27   

                           comment  text  coder  indirect  vague  past  \
cro cro_sub_type_combined                                                
OP  PRODUCTS                   114   423    423       423    423   423   
    RESILIENCE                 158   423    423       423    423   423   
PR  ACUTE                       88   113    113       113    113   113   
    CHRON                       17    21     21        21     21    21   
TR  MARKET                      33    51     51        51     51    51   
    POLICY                      36   109    109       109    109   109   
    REPUTATION                  18    29     29        29     29    29   

                           keyword  span_id  neg_type  
cro cro_sub_type_combined                              
OP  PRODUCTS                    10       31         0  
    RESILIENCE                  18       28         0  
PR  ACUTE                       12        7         0  
    CHRON                       10        0         0  
TR  MARKET                      19        0         0  
    POLICY                      21        7         0  
    REPUTATION                   8        3         0

In [5]:
df_test.groupby(['cro', 'cro_sub_type_combined']).count()

report_id  cro_sub_type  page  paragraph_no  label  \
cro cro_sub_type_combined                                                       
OP  PRODUCTS                      18            18    18            18     18   
    RESILIENCE                     4             4     4             4      4   
PR  ACUTE                         20            20    20            20     20   
    CHRON                          6             6     6             6      6   
    MARKET                         1             1     1             1      1   
TR  MARKET                        11            11    11            11     11   
    POLICY                        19            19    19            19     19   
    REPUTATION                     7             7     7             7      7   

                           comment  text  coder  indirect  vague  past  \
cro cro_sub_type_combined                                                
OP  PRODUCTS                     0    18     18        18     18    18   
    RESILIENCE                   0     4      4         4      4     4   
PR  ACUTE                       10    20     20        20     20    20   
    CHRON                        4     6      6         6      6     6   
    MARKET                       1     1      1         1      1     1   
TR  MARKET                       5    11     11        11     11    11   
    POLICY                       3    19     19        19     19    19   
    REPUTATION                   1     7      7         7      7     7   

                           keyword  span_id  neg_type  
cro cro_sub_type_combined                              
OP  PRODUCTS                     0        0         0  
    RESILIENCE                   0        0         0  
PR  ACUTE                        5        4         0  
    CHRON                        4        0         0  
    MARKET                       1        0         0  
TR  MARKET                       4        0         0  
    POLICY                       3        0         0  
    REPUTATION                   1        0         0

In [34]:
df_test.groupby("neg_type", dropna=False).count()

,report_id,cro,cro_sub_type,page,paragraph_no,label,comment,text,coder,indirect,vague,past,keyword,span_id,cro_sub_type_combined
neg_type,,,,,,,,,,,,,,,
hot,2162,0,0,2162,2162,0,0,2162,2162,0,0,0,0,0,0
weak,25970,0,0,25970,25970,0,0,25970,25970,0,0,0,0,0,0
NaN,94,94,88,94,94,94,32,94,94,94,94,94,26,4,86


In [36]:
result = []

pos_labels = df_test.query("neg_type != neg_type")
for index, row in df_test.query("neg_type == ['weak', 'hot']").iterrows():
    is_in_pos = any(row.text in text for text in pos_labels.text)
    if is_in_pos:
        index = [idx for idx, text in enumerate(pos_labels.text) if row.text in text][0]
        pos_row = pos_labels.iloc[index]
        if pos_row.report_id == row.report_id and pos_row.page == row.page:
            result.append(
                [row.report_id, row.page, row.paragraph_no, row.text, row.neg_type, pos_row.paragraph_no, pos_row.text, pos_row.cro, pos_row.cro_sub_type, "test"]
            )
no_in_test = len(result)
print(f"Found {no_in_test} overlappings in the test set...")

Found 0 overlappings in the test set...


In [37]:
result = []
pos_labels = df_train.query("neg_type != neg_type")
for index, row in df_train.query("neg_type == ['weak', 'hot']").iterrows():
    is_in_pos = any(row.text in text for text in pos_labels.text)
    if is_in_pos:
        index = [idx for idx, text in enumerate(pos_labels.text) if row.text in text][0]
        pos_row = pos_labels.iloc[index]
        if pos_row.report_id == row.report_id and pos_row.page == row.page:
            result.append(
                [row.report_id, row.page, row.paragraph_no, row.text, row.neg_type, pos_row.paragraph_no, pos_row.text, pos_row.cro, pos_row.cro_sub_type, "training"]
            )
print(f"Found {len(result) - no_in_test} overlappings in training set....")

Found 106 overlappings in training set....


In [38]:
df_result = pd.DataFrame(result, columns=["report_id", "neg_page", "neg_paragraph_no", "neg_text", "neg_type", "pos_paragraph_no", 'pos_text', 'pos_cro', 'pos_cro_sub_type', 'labelling_dataset'])

In [39]:
df_result

,report_id,neg_page,neg_paragraph_no,neg_text,neg_type,pos_paragraph_no,pos_text,pos_cro,pos_cro_sub_type,labelling_dataset
0,de_sap-AR_2009,134,8,\nTerrorist attacks and other acts of violence...,hot,7,"Social and political instability caused, for e...",PR,ACUTE,training
1,de_sap-AR_2009,138,1,"\nFor SAP, sustainability is a standard that g...",hot,0,SAP’s sustainability strategy may be difficult...,TR,REPUT,training
2,gb_vodafone_grp-AR_2011,48,12,\nWe are dependent on the secure operation of ...,hot,7,Our business could be adversely affected by di...,PR,ACUTE,training
3,es_telefonica-AR_2016,170,22,\nPower: Technology update for power units and...,hot,21,PsF (Power saving Features): Activation of ene...,OP,EFFI,training
4,es_telefonica-AR_2018,28,1,"\nScope 2: Scope 2 emissions, from power consu...",hot,0,Scope 1: Our Scope 1 emissions come from two m...,OP,ENERGY,training
...,...,...,...,...,...,...,...,...,...,...
101,de_basf-AR_2007,102,9,"O), ammonia and other\ninorganic compounds suc...",weak,7,"O), ammonia and other\ninorganic compounds suc...",OP,RESILI,training
102,de_basf-AR_2007,102,10,"\nIn 2007, emissions to air from BASF’s chemic...",weak,7,"O), ammonia and other\ninorganic compounds suc...",OP,RESILI,training
103,de_basf-AR_2007,103,7,"\nIn 2007, BASF’s water requirements worldwide...",weak,6,We had set ourselves the goal of reducing emis...,OP,EFFI,training
104,de_basf-AR_2012,41,2,\nThe number and quality of our patents attest...,weak,1,Our research pipeline included approximately 3...,OP,PRODUCTS,training


In [40]:
df_result.query("labelling_dataset == 'training'").to_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training_Overlapping.pkl")